In [46]:
import trino
import pandas as pd
import warnings
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

# you need not change these
HOST = 'starburst.g8s-data-platform-prod.glovoint.com'
PORT = 443

conn_details = {
    'host': HOST,
    'port': PORT,
    'http_scheme': 'https',
    'auth': trino.auth.OAuth2Authentication()
}

In [47]:
def get_recurrent_customers_for_period(period_start_date, period_end_date):
    sql_query = f"""
    WITH nc as (
    SELECT DISTINCT customer_id
    FROM delta.central_order_descriptors_odp.order_descriptors_v2
    WHERE order_country_code = 'PL'
      AND order_final_status = 'DeliveredStatus'
      AND order_parent_relationship_type IS NULL
      AND order_is_first_delivered_order = true
      AND order_started_local_at < DATE '{period_end_date}' and order_started_local_at >= DATE '{period_start_date}'
),
    rc AS (
    SELECT DISTINCT customer_id
    FROM delta.central_order_descriptors_odp.order_descriptors_v2
    WHERE order_country_code = 'PL'
      AND order_final_status = 'DeliveredStatus'
      AND order_parent_relationship_type IS NULL
      AND order_started_local_at < DATE '{period_end_date}' and order_started_local_at >= DATE '{period_start_date}'
    EXCEPT
    SELECT * FROM nc
),
    first_monthly_order_date_rc AS (
    SELECT customer_id, date(MIN(order_started_local_at)) AS monthly_first_order_date
    FROM delta.central_order_descriptors_odp.order_descriptors_v2
    WHERE customer_id IN (SELECT customer_id FROM rc)
    AND order_started_local_at < DATE '{period_end_date}' and order_started_local_at >= DATE '{period_start_date}'
    GROUP BY customer_id
),
    last_order_before_first AS (
    SELECT a.customer_id, date(MAX(b.order_started_local_at)) AS last_order_date
    FROM first_monthly_order_date_rc a
    LEFT JOIN delta.central_order_descriptors_odp.order_descriptors_v2 b ON a.customer_id = b.customer_id
    WHERE b.order_started_local_at < a.monthly_first_order_date
    GROUP BY a.customer_id
),
    rc_categorization AS (
    SELECT
        a.customer_id,
        a.monthly_first_order_date,
        b.last_order_date,
        CASE
            WHEN DATE_DIFF('day', b.last_order_date, a.monthly_first_order_date) <= 28 THEN 'Ongoing'
            WHEN DATE_DIFF('day', b.last_order_date, a.monthly_first_order_date) > 28 THEN 'Reactivated'
        END AS customer_status
    FROM first_monthly_order_date_rc a
    JOIN last_order_before_first b ON a.customer_id = b.customer_id
),
    ongoing_customers AS (
        SELECT customer_id
        FROM rc_categorization
        WHERE customer_status = 'Ongoing'
),
    reactivated_customers AS (
        SELECT customer_id
        FROM rc_categorization
        WHERE customer_status = 'Reactivated'
)

SELECT 'Ongoing' as status, customer_id FROM ongoing_customers
UNION ALL
SELECT 'Reactivated' as status, customer_id FROM reactivated_customers"""  
    with trino.dbapi.connect(**conn_details) as conn:
        df = pd.read_sql_query(sql_query, conn)
    return df

# with warnings.catch_warnings():
#     warnings.simplefilter("ignore", category=UserWarning)
#     print("Running the query...")
#     recurrent_customers = get_recurrent_customers_for_period(START, END)
#     print("Completed")

In [24]:
print(recurrent_customers)

             status  customer_id
0       Reactivated    114768334
1       Reactivated     46270496
2       Reactivated    134224443
3       Reactivated    108782107
4       Reactivated    106586057
...             ...          ...
623033      Ongoing    171848478
623034      Ongoing    171794236
623035      Ongoing    108452002
623036      Ongoing    170966828
623037      Ongoing    165845529

[623038 rows x 2 columns]


In [69]:

# Dictionary to store dataframes
all_customers = {}

# Set current_date to the last day of the previous month
current_date = datetime.now().replace(day=1) - relativedelta(days=1)

# Loop through the past 13 months

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    for i in tqdm(range(2), desc="Processing queries"):
        end_date = current_date - relativedelta(months=i)
        start_date = end_date.replace(day=1)  # Set to the first day of the month
        start_date_str = str(start_date.strftime('%Y-%m-%d'))
        end_date_str = str(end_date.strftime('%Y-%m-%d'))
        print(start_date_str)
        print(end_date_str)
        df = get_recurrent_customers_for_period(start_date_str, end_date_str)
        all_customers[f"{start_date_str} to {end_date_str}"] = df
    print("Completed processing all periods.")



Processing queries:   0%|          | 0/2 [00:00<?, ?it/s]

2024-07-01
2024-07-31


Processing queries:  50%|█████     | 1/2 [01:27<01:27, 87.32s/it]

2024-06-01
2024-06-30


Processing queries: 100%|██████████| 2/2 [02:39<00:00, 79.63s/it]

Completed processing all periods.


In [70]:
print(all_customers)

{'2024-07-01 to 2024-07-31':              status  customer_id
0       Reactivated     35089135
1       Reactivated     30452951
2       Reactivated     45482466
3       Reactivated     44788389
4       Reactivated     12607340
...             ...          ...
614166      Ongoing    172244985
614167      Ongoing    172281838
614168      Ongoing    172293937
614169      Ongoing    148590601
614170      Ongoing    169299861

[614171 rows x 2 columns], '2024-06-01 to 2024-06-30':              status  customer_id
0       Reactivated     11438023
1       Reactivated     53095306
2       Reactivated     28752807
3       Reactivated     50677934
4       Reactivated     47693951
...             ...          ...
666429      Ongoing    137008451
666430      Ongoing     66324834
666431      Ongoing     51265775
666432      Ongoing    149597851
666433      Ongoing    162010045

[666434 rows x 2 columns]}
